## Faster opnorm for IntervalMatrix

https://github.com/JuliaReach/IntervalMatrices.jl/pull/125

In [27]:
using Revise, LazySets, IntervalMatrices, LinearAlgebra
using BenchmarkTools

In [22]:
#   26.181 μs (8 allocations: 234.64 KiB)
function opnorm1(A::IntervalMatrix, p::Real=Inf)
    if p == Inf || p == 1
        return LinearAlgebra.opnorm(max.(abs.(inf(A)), abs.(sup(A))), p)
    else
        error("the interval matrix norm for this value of p=$p is not implemented")
    end
end

opnorm1 (generic function with 2 methods)

In [23]:
#   26.080 μs (8 allocations: 234.64 KiB)
function opnorm2(A::IntervalMatrix, p::Real=Inf)
    if p == Inf || p == 1
        return LinearAlgebra.opnorm(max.((-).(inf(A)), sup(A)), p)
    else
        error("the interval matrix norm for this value of p=$p is not implemented")
    end
end

#   33.279 μs (10 allocations: 312.84 KiB)
function opnorm2b(A::IntervalMatrix, p::Real=Inf)
    if p == Inf || p == 1
        return LinearAlgebra.opnorm(max.(-inf(A), sup(A)), p)
    else
        error("the interval matrix norm for this value of p=$p is not implemented")
    end
end

opnorm2b (generic function with 2 methods)

In [154]:
function opnorm3(A::IntervalMatrix, p::Real=Inf)
    if p == Inf
        return _opnorm_inf_a(A)
    elseif p == 1
        return _opnorm_1(A)
    else
        error("the interval matrix norm for this value of p=$p is not implemented")
    end
end

function _opnorm_1(A) end

function _opnorm_inf_a(A::IntervalMatrix{N}) where {N}
    res = zero(N)
    m, n = size(A)
    @inbounds @simd for i in 1:m
        acc = zero(N)
        for j in 1:m
            x = A[i, j]
            acc += max(-inf(x), sup(x))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

function _opnorm_inf_b(A::IntervalMatrix{N}) where {N}
    res = zero(N)
    m, n = size(A)
    @inbounds @simd for i in 1:m
        acc = zero(N)
        for j in 1:m
            x = A[i, j]
            acc += max(abs(inf(x)), abs(sup(x)))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

function _opnorm_inf_c(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    acc = zeros(N, m)
    @inbounds @simd for j in 1:m
        for i in 1:n
            x = A[i, j]
            acc[i] += max(-inf(x), sup(x))
        end
    end
    return maximum(acc)
end

function _opnorm_inf_d(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    acc = zeros(N, m)
    @inbounds @simd for j in 1:m
        for i in 1:n
            x = A[i, j]
            acc[i] += max(abs(inf(x)), abs(sup(x)))
        end
    end
    return maximum(acc)
end

function _opnorm_inf_e(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    res = zero(N)
    At = transpose(A)
    @inbounds @simd for j in 1:m
        acc = zero(N)
        for i in 1:n
            x = At[i, j]
            acc += max(abs(inf(x)), abs(sup(x)))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

function _opnorm_inf_f(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    res = zero(N)
    At = transpose(A)
    @inbounds @simd for j in 1:m
        acc = zero(N)
        for i in 1:n
            x = At[i, j]
            acc += max(-inf(x), sup(x))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

function _opnorm_inf_g(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    res = zero(N)
    At = transpose(A)
    @inbounds @simd for j in 1:m
        acc = zero(N)
        for i in 1:n
            x = At[i, j]
            acc += max(abs(inf(x)), abs(sup(x)))
            #acc += max(-inf(x), sup(x))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

#=
Results:
  48.442 μs (0 allocations: 0 bytes)
  18.663 μs (0 allocations: 0 bytes)
  49.184 μs (1 allocation: 896 bytes)
  20.629 μs (1 allocation: 896 bytes)
  18.683 μs (0 allocations: 0 bytes) # best is version e
  50.652 μs (0 allocations: 0 bytes)
=#

_opnorm_inf_g (generic function with 1 method)

In [138]:
A = rand(IntervalMatrix, 100);

In [139]:
opnorm1(A)

127.15752594236561

In [140]:
opnorm2(A)

127.15752594236561

In [141]:
opnorm2b(A)

127.15752594236561

In [142]:
@show _opnorm_inf_a(A)
@show _opnorm_inf_b(A)
@show _opnorm_inf_c(A)
@show _opnorm_inf_d(A)
@show _opnorm_inf_e(A)
@show _opnorm_inf_f(A)

_opnorm_inf_a(A) = 127.15752594236561
_opnorm_inf_b(A) = 127.15752594236561
_opnorm_inf_c(A) = 127.15752594236561
_opnorm_inf_d(A) = 127.15752594236561
_opnorm_inf_e(A) = 127.15752594236561
_opnorm_inf_f(A) = 127.15752594236561


127.15752594236561

In [143]:
@btime opnorm1($A)

  26.181 μs (8 allocations: 234.64 KiB)


127.15752594236561

In [144]:
@btime opnorm2($A)

  26.080 μs (8 allocations: 234.64 KiB)


127.15752594236561

In [145]:
@btime opnorm2b($A)

  33.279 μs (10 allocations: 312.84 KiB)


127.15752594236561

In [146]:
@btime _opnorm_inf_a($A)
@btime _opnorm_inf_b($A)
@btime _opnorm_inf_c($A)
@btime _opnorm_inf_d($A)
@btime _opnorm_inf_e($A) # fastest version
@btime _opnorm_inf_f($A)

  48.442 μs (0 allocations: 0 bytes)
  18.663 μs (0 allocations: 0 bytes)
  49.184 μs (1 allocation: 896 bytes)
  20.629 μs (1 allocation: 896 bytes)
  18.683 μs (0 allocations: 0 bytes)
  50.652 μs (0 allocations: 0 bytes)


127.15752594236561

In [155]:
@btime _opnorm_inf_g($A)

  18.817 μs (0 allocations: 0 bytes)


127.15752594236561

Result: `max(|inf(i)|, |sup(i)|) == max(-inf(i), sup(i))` for interval `i`

In [113]:
f(x) = max(abs(inf(x)), abs(sup(x)))
g(x) = max(-inf(x), sup(x))

g (generic function with 1 method)

In [114]:
x = [rand(IntervalMatrices.Interval) for _ in 1:1000];

In [115]:
@btime f.($x);

  1.633 μs (1 allocation: 7.94 KiB)


In [116]:
@btime g.($x);

  1.019 μs (1 allocation: 7.94 KiB)


In [147]:
(26 - 18.6)/26 * 100

28.461538461538456

In [149]:
(26.181 - 26.080)/(26.181) * 100

0.38577594438716106

## Pull request

In [1]:
using Revise, IntervalMatrices, Test, BenchmarkTools

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/IntervalMatrices/VerWp.ji for IntervalMatrices [5c1f47dc-42dd-5697-8aaa-4d102d140ba9]
└ @ Base loading.jl:1240


In [4]:
# The operator norm in the infinity norm corresponds to the
# maximum absolute value row-sum. Julia has column-major
# ordering so for efficiency we iterate over the transpose of A.
function _opnorm_inf_e(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    res = zero(N)
    At = transpose(A)
    @inbounds @simd for j in 1:m
        acc = zero(N)
        for i in 1:n
            x = At[i, j]
            acc += max(abs(inf(x)), abs(sup(x)))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

_opnorm_inf_e (generic function with 1 method)

In [13]:
# The operator norm in the 1-norm corresponds to the
# maximum absolute value column-sum.
function _opnorm_1_e(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    res = zero(N)
    @inbounds @simd for j in 1:m
        acc = zero(N)
        for i in 1:n
            x = A[i, j]
            acc += max(abs(inf(x)), abs(sup(x)))
        end
        if acc > res
            res = acc
        end
    end
    return res
end

function _opnorm_1_e_v2(A::IntervalMatrix{N}) where {N}
    m, n = size(A)
    res = zero(N)
    @inbounds @simd for j in 1:m
        acc = zero(N)
        for i in 1:n
            x = A[i, j]
            acc += max(abs(inf(x)), abs(sup(x)))
        end
        (acc > res) && (res = acc)
    end
    return res
end

_opnorm_1_e_v2 (generic function with 1 method)

In [2]:
M = rand(IntervalMatrix, 100);

In [3]:
opnorm(M, Inf)

128.9128489817419

In [6]:
@btime opnorm($M, Inf)

  23.559 μs (8 allocations: 234.64 KiB)


128.9128489817419

In [7]:
@btime _opnorm_inf_e($M)

  18.804 μs (0 allocations: 0 bytes)


128.9128489817419

In [9]:
@btime opnorm($M, 1)

  24.221 μs (8 allocations: 234.64 KiB)


130.11834643451667

In [8]:
@btime _opnorm_1_e($M)

  17.752 μs (0 allocations: 0 bytes)


130.11834643451667

In [15]:
@btime _opnorm_1_e_v2($M)

  17.887 μs (0 allocations: 0 bytes)


130.11834643451667

In [29]:
M = rand(IntervalMatrix, 3)

3×3 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.584893, -0.246486]  [-0.101462, 0.981223]  [-1.23482, -0.0502436]
 [-0.144607, 0.789303]   [-0.271674, 1.92081]   [-0.989563, 0.0397058]
 [-1.43055, 1.16695]     [-1.253, -0.372325]     [0.488512, 0.695137] 

In [32]:
@btime opnorm($M, 1)

  21.029 ns (0 allocations: 0 bytes)


4.155029632241096

In [ ]:
using LinearAlgebra

In [30]:
@btime opnorm1($M, 1)

  160.303 ns (6 allocations: 528 bytes)


4.155029632241096

In [31]:
@btime opnorm2($M, 1)

  120.207 ns (5 allocations: 512 bytes)


4.155029632241096

## See unit_halfspace.jl

The following example produces an unexpected output (the commented test). I think that the result of the elimination should be `y = 0, x <= 0`.

In [6]:
using LazySets # mforets/refactor_linear_map_hrep
using Polyhedra, CDDLib

N = Float64
H = HalfSpace(N[1, -1], N(0)) # x <= y

HalfSpace{Float64,Array{Float64,1}}([1.0, -1.0], 0.0)

In [7]:
M = N[1 0; 0 0] # project variable y -> 0, and identity on x, x -> x

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  0.0

In [9]:
linear_map(M, H).constraints # this is y = 0, and x ∈ [-inf, inf]

2-element Array{HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1},1}:
 HalfSpace{Float64,Array{Float64,1}}([-0.0, -1.0], 0.0)
 HalfSpace{Float64,Array{Float64,1}}([0.0, 1.0], -0.0) 

## Interval Matrix

## Elimination can't handle data with different array types

# NVerif

In [7]:
using Revise, LazySets, BenchmarkTools

In [8]:
h = rand(Hyperrectangle, dim=5);
p = convert(HPolytope, h);
v = rand(5);

In [21]:
@btime y = translate($p, $v);

  1.283 μs (46 allocations: 2.06 KiB)


In [18]:
y = translate(p, v);
y

HPolytope{Float64}(HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[HalfSpace{Float64,Array{Float64,1}}([1.0, 0.0, 0.0, 0.0, 0.0], 1.0925021749226569), HalfSpace{Float64,Array{Float64,1}}([0.0, 1.0, 0.0, 0.0, 0.0], 1.085247820435545), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 1.0, 0.0, 0.0], 0.2378793282299262), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 0.0, 1.0, 0.0], 3.758010061500308), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, 1.0], 1.3046230810839647), HalfSpace{Float64,Array{Float64,1}}([-1.0, 0.0, 0.0, 0.0, 0.0], -0.5384077723303445), HalfSpace{Float64,Array{Float64,1}}([0.0, -1.0, 0.0, 0.0, 0.0], -0.718445792008757), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, -1.0, 0.0, 0.0], 0.7899252186764041), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 0.0, -1.0, 0.0], -1.7473476724551364), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, -1.0], 0.9304067235918292)])

In [20]:
@which translate(p, v)

translate(P::PT, v::AbstractArray{N,1}) where {N<:Real, PT<:Union{HPolyhedron{N}, HPolytope{N}}} in LazySets at /home/mforets/.julia/dev/LazySets/src/Sets/HPolyhedron.jl:415

In [16]:
# from neural verif
function translate0(v::Vector, H::HPolytope)
    # translate each halfpsace according to:
    # a⋅(x-v) ≤ b  ⟶  a⋅x ≤ b+a⋅v
    C, d = tosimplehrep(H)
    return HPolytope(C, d+C*v)
end

translate0 (generic function with 1 method)

In [19]:
@btime translate0($v, $p)

  1.354 μs (60 allocations: 3.56 KiB)


HPolytope{Float64}(HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1}[HalfSpace{Float64,Array{Float64,1}}([1.0, 0.0, 0.0, 0.0, 0.0], 1.0925021749226569), HalfSpace{Float64,Array{Float64,1}}([0.0, 1.0, 0.0, 0.0, 0.0], 1.085247820435545), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 1.0, 0.0, 0.0], 0.2378793282299262), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 0.0, 1.0, 0.0], 3.758010061500308), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, 1.0], 1.3046230810839647), HalfSpace{Float64,Array{Float64,1}}([-1.0, 0.0, 0.0, 0.0, 0.0], -0.5384077723303445), HalfSpace{Float64,Array{Float64,1}}([0.0, -1.0, 0.0, 0.0, 0.0], -0.718445792008757), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, -1.0, 0.0, 0.0], 0.7899252186764041), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 0.0, -1.0, 0.0], -1.7473476724551364), HalfSpace{Float64,Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, -1.0], 0.9304067235918292)])

In [22]:
h

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.5880491987203353, 0.8931753898170605, -0.5683002100948344, 1.7942350864249061, -0.07870194545014124], [0.2770472012961561, 0.18340101421339394, 0.5139022734531652, 1.0053311945225858, 1.117514902337897])

In [24]:
translate0(v::Vector, H::Hyperrectangle)  = Hyperrectangle(H.center .+ v, H.radius)

translate0 (generic function with 2 methods)

In [26]:
@btime translate0($v, $h)

  40.763 ns (2 allocations: 160 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.8154549736265007, 0.901846806222151, -0.276022945223239, 2.752678866977722, 0.18710817874606772], [0.2770472012961561, 0.18340101421339394, 0.5139022734531652, 1.0053311945225858, 1.117514902337897])

In [32]:
@btime translate($h, $v)

  33.859 ns (2 allocations: 160 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.8154549736265007, 0.901846806222151, -0.276022945223239, 2.752678866977722, 0.18710817874606772], [0.2770472012961561, 0.18340101421339394, 0.5139022734531652, 1.0053311945225858, 1.117514902337897])

In [ ]:
#=
# translate(v::Vector, H::Hyperrectangle)  = Hyperrectangle(H.center .+ v, H.radius)
translate(v::Vector, V::AbstractPolytope) = tohrep(VPolytope([x+v for x in vertices_list(V)]))
=#

Both implementations are good:

```julia
function affine_map(layer::Layer, input::AbstractPolytope)
    W, b = layer.weights, layer.bias
    P = linear_map(W, input, algorithm="vrep")
    return tohrep(translate(P, b))
end
```

```julia
function affine_map(layer::Layer, input::AbstractPolytope)
    W, b = layer.weights, layer.bias
    P = affine_map(W, tovrep(input), b)
    return tohrep(P)
end
```

### Activating/deactivating assertions

```julia
using NeuralVerification, LazySets, GLPKMathProgInterface
using Test

# comment to re-enable assertion checks
LazySets.deactivate_assertions()
LazySets.Assertions.deactivate_assertions(NeuralVerification)
```

```julia
[mforets@localhost LazySets]$ git diff
diff --git a/src/Interfaces/AbstractPolyhedron_functions.jl b/src/Interfaces/AbstractPolyhedron_functions.jl
index 9303f1d6..051f726d 100644
--- a/src/Interfaces/AbstractPolyhedron_functions.jl
+++ b/src/Interfaces/AbstractPolyhedron_functions.jl
@@ -724,9 +724,15 @@ function _linear_map_hrep(M::AbstractMatrix{N}, P::AbstractPolyhedron{N},
     method = algo.method


     # extend the polytope storing the y variables first
-    Ax_leq_b = [Polyhedra.HalfSpace(vcat(zeros(N, m), c.a), c.b) for c in constraints_list(P)]
-    y_eq_Mx = [Polyhedra.HyperPlane(vcat(-id_m[i, :], M[i, :]), zero(N)) for i in 1:m]
-
+    Ax_leq_b = [Polyhedra.HalfSpace(Vector(vcat(zeros(N, m), c.a)), c.b) for c in constraints_list(P)]
+    y_eq_Mx = [Polyhedra.HyperPlane(Vector(vcat(-id_m[i, :], M[i, :])), zero(N)) for i in 1:m]
+    println(" = = enter = = ")
+    @show typeof(M)
+    @show typeof(P)
+    @show typeof(P.constraints[1])
+    @show typeof(Ax_leq_b)
+    @show typeof(y_eq_Mx)
+    println(" = = end = =")
     Phrep = Polyhedra.hrep(y_eq_Mx, Ax_leq_b)
     Phrep_cdd = polyhedron(Phrep, backend)
     Peli_block = Polyhedra.eliminate(Phrep_cdd, (m+1):(m+n), method)
```

## Interval matrix power (cont)

In [1]:
using Revise, IntervalMatrices, Test, BenchmarkTools

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/IntervalMatrices/VerWp.ji for IntervalMatrices [5c1f47dc-42dd-5697-8aaa-4d102d140ba9]
└ @ Base loading.jl:1240


In [23]:
B = IntervalMatrix([2.0±0 0.0±0; 0.0±0 3.0±0])

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [2, 2]  [0, 0]
 [0, 0]  [3, 3]

In [24]:
p = IntervalMatrixPower(B)

IntervalMatrixPower{Float64}(Interval{Float64}[[2, 2] [0, 0]; [0, 0] [3, 3]], Interval{Float64}[[2, 2] [0, 0]; [0, 0] [3, 3]], 1)

In [25]:
for i in 1:20
    increment!(p)
end

In [26]:
get(p)

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
     [2.09715e+06, 2.09716e+06]  [0, 0]                        
 [0, 0]                              [1.04603e+10, 1.04604e+10]

In [ ]:
julia> 
2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [2, 2]  [0, 0]
 [0, 0]  [3, 3]

julia> C = [2.0 0.0; 0.0 3.0]
2×2 Array{Float64,2}:
 2.0  0.0
 0.0  3.0

julia> 
2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [1.89999, 2.10001]    [-0.100001, 0.100001]
 [-0.100001, 0.100001]   [2.89999, 3.10001]  

In [28]:
D = IntervalMatrix([2.0±0.1 0.0±0.1; 0.0±0.1 3.0±0.1])

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [1.89999, 2.10001]    [-0.100001, 0.100001]
 [-0.100001, 0.100001]   [2.89999, 3.10001]  

In [77]:
D = rand(IntervalMatrix, 3)
p = IntervalMatrixPower(D)

IntervalMatrixPower{Float64}(Interval{Float64}[[-0.988596, 0.666005] [-2.04443, -1.49919] [0.755732, 1.97913]; [-0.691819, 0.949111] [0.510271, 0.671536] [-0.272319, 2.33641]; [0.760882, 1.41149] [-1.79644, 1.18148] [-1.419, 1.22011]], Interval{Float64}[[-0.988596, 0.666005] [-2.04443, -1.49919] [0.755732, 1.97913]; [-0.691819, 0.949111] [0.510271, 0.671536] [-0.272319, 2.33641]; [0.760882, 1.41149] [-1.79644, 1.18148] [-1.419, 1.22011]], 1)

In [78]:
p = IntervalMatrixPower(D)
for i in 1:7
    increment!(p)
end
get(p)

3×3 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-16241.4, 30680.4]  [-29201.1, 33451.1]  [-46174.6, 33206.5]
 [-19628.1, 20965.8]  [-16634.1, 22917.2]  [-31827.9, 29752.1]
 [-27525.9, 19993.3]  [-30084.9, 26631.1]  [-19112.4, 41949]  

In [79]:
index(p)

8

In [80]:
D^8

3×3 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-24855.9, 31784.1]  [-36509.1, 36418.1]  [-50520, 42480.4]         
 [-22140.7, 24141]    [-27870.2, 29220]           [-37055.6, 36207.4]
 [-28623.6, 25474.5]  [-34528.9, 32058.3]         [-37974.6, 45755.3]

In [ ]:
julia> D^16
2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-664454, 874628]                          [-7.49338e+06, 7.49338e+06]
               [-7.49338e+06, 7.49338e+06]   [2.32098e+07, 7.58084e+07]

In [81]:
M = rand(IntervalMatrix, 3)

3×3 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.554135, 0.104259]   [0.70749, 1.29983]     [-0.807291, 1.27384] 
 [-1.65039, -0.909366]  [-0.211832, 0.0715744]  [-0.0695552, 1.42866]
 [-0.18047, 0.17225]    [-1.27171, 0.615581]    [-0.944456, 0.986335]

In [82]:
square(M) * M

3×3 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-2.90501, 6.19197]  [-7.48135, 3.01396]   [-9.82813, 6.9125] 
 [-1.94722, 7.52009]  [-4.61229, 7.5766]   [-10.5825, 5.87088] 
 [-4.48309, 3.68287]  [-4.64961, 6.0171]    [-6.18858, 7.48208]

In [84]:
M * square(M)

3×3 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-2.87263, 6.29718]  [-7.85806, 3.97288]  [-9.28589, 7.4433] 
 [-2.03383, 7.42075]  [-4.61229, 7.5766]   [-8.88396, 7.05476]
 [-4.56732, 3.69517]  [-4.48972, 7.18176]  [-6.22096, 7.37686]

In [85]:
(M*M)*M

3×3 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-2.9678, 6.27816]   [-7.95645, 3.01396]  [-10.1988, 7.26938]
 [-1.94722, 7.54511]  [-4.61337, 7.57981]  [-10.6042, 5.87088]
 [-4.62221, 3.92446]  [-4.66953, 7.21009]   [-7.16365, 8.2515]

In [86]:
M^3

3×3 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-2.87866, 6.3292]  [-7.92793, 3.97288]  [-10.6469, 7.4433]  
 [-2.05572, 7.5161]  [-4.66963, 7.57981]  [-10.734, 7.05476]  
 [-4.57727, 3.7056]  [-4.49905, 7.20104]   [-7.19655, 8.20046]

## Transmission line

In [13]:
using MathematicalSystems, LinearAlgebra, SparseArrays

function tline(;η=2, R₀=1.00, Rd₀=10.0, L₀=1e-10, C₀=1e-13 * 4.00, scale=1e-9)
    A₁₁ = zeros(η, η)
    A₁₂ = Bidiagonal(fill(-1/C₀, η), fill(1/C₀, η-1), :U)
    A₂₁ = Bidiagonal(fill(1/L₀, η), fill(-1/L₀, η-1), :L)
    A₂₂ = Diagonal(vcat(-Rd₀/L₀, fill(-R₀/L₀, η-1)))
    A  = [A₁₁ A₁₂; A₂₁ A₂₂] .* scale
    return A
end

tline (generic function with 1 method)

In [18]:
A = tline()

4×4 SparseMatrixCSC{Float64,Int64} with 8 stored entries:
  [3, 1]  =  10.0
  [4, 1]  =  -10.0
  [4, 2]  =  10.0
  [1, 3]  =  -2500.0
  [3, 3]  =  -100.0
  [1, 4]  =  2500.0
  [2, 4]  =  -2500.0
  [4, 4]  =  -10.0

In [19]:
using IntervalMatrices

In [23]:
IntervalMatrix(Matrix(A))

MethodError: MethodError: no method matching IntervalMatrix(::Array{Float64,2})
Closest candidates are:
  IntervalMatrix(::MT<:AbstractArray{T,2}, !Matched::MT<:AbstractArray{T,2}) where {T, MT<:AbstractArray{T,2}} at /home/mforets/.julia/dev/IntervalMatrices/src/matrix.jl:129
  IntervalMatrix(!Matched::MT<:AbstractArray{IT<:Interval{T},2}) where {T, IT<:Interval{T}, MT<:AbstractArray{IT,2}} at /home/mforets/.julia/dev/IntervalMatrices/src/matrix.jl:71
  IntervalMatrix(!Matched::UniformScaling{Interval{T}}, !Matched::Integer) where T at /home/mforets/.julia/dev/IntervalMatrices/src/matrix.jl:84
  ...

In [37]:
# k = a^2 + b
k = 13
a = floor(Int, sqrt(k))
b = k - a^2

4

In [38]:
a

3

In [39]:
b

4

# MSys macros

- https://github.com/JuliaReach/MathematicalSystems.jl/pull/155

In [1]:
using Revise, MathematicalSystems

In [2]:
A = rand(2, 2);
B = rand(2, 1);

In [4]:
@system(x⁺ = Ax + Bw + Dw, x ∈ X, w ∈ U1, w ∈ W1, noise=w, input=w)

ArgumentError: ArgumentError: input and noise variables have the same name `w`

In [14]:
@system(x' = Ax + Bw, 

ArgumentError: ArgumentError: the entry (:A, :D) does not match a `MathematicalSystems.jl` structure

In [10]:
@system(x' = Ax + Bw)

ArgumentError: ArgumentError: the entry (:A, :D) does not match a `MathematicalSystems.jl` structure

In [15]:
@system(u' = Au)

LinearContinuousSystem{Float64,Array{Float64,2}}([0.9274312900638151 0.86384328799834; 0.8100434712797464 0.10382888327222162])

In [19]:
@system(x' = Ax + Bu)

LinearControlContinuousSystem{Float64,Array{Float64,2},Array{Float64,2}}([0.9274312900638151 0.86384328799834; 0.8100434712797464 0.10382888327222162], [0.4692293797466649; 0.631866585230791])

In [22]:
@system(x' = Ax + Bw, input=w)

LinearControlContinuousSystem{Float64,Array{Float64,2},Array{Float64,2}}([0.9274312900638151 0.86384328799834; 0.8100434712797464 0.10382888327222162], [0.4692293797466649; 0.631866585230791])

In [23]:
D = rand(2, 1)

2×1 Array{Float64,2}:
 0.06833206453281249
 0.04503291267148146

In [26]:
@system(x' = Ax + Bu + Dw, input=u, noise=w)

ArgumentError: ArgumentError: the entry (:A, :B, :D) does not match a `MathematicalSystems.jl` structure

In [46]:
X = rand(2); U1 = rand(2); W1 = rand(2);

In [40]:
@system(x' = Ax + Bx, input=x)

LoadError: AssertionError: found more than one compatible system type

In [36]:
@system(x' = Ax + Bx, input=x)

2×2 Array{Float64,2}:
 0.927431  0.863843
 0.810043  0.103829

In [37]:
B

2×1 Array{Float64,2}:
 0.4692293797466649
 0.631866585230791 

## Extend @system for initial-value problems

https://github.com/JuliaReach/MathematicalSystems.jl/issues/137

In [1]:
using Revise, MathematicalSystems, LazySets

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/MathematicalSystems/6oLdk.ji for MathematicalSystems [d14a8603-c872-5ed3-9ece-53e0e82e39da]
└ @ Base loading.jl:1240


In [2]:
using MacroTools
using MacroTools: @capture

In [3]:
ex = :(x(0) ∈ Interval(-1.0, 1.0))
@capture(ex, x_(0) ∈ X0_)

true

In [4]:
@system(x' = -x, t(0) ∈ Interval(-1.0, 1.0)) # OK

ArgumentError: ArgumentError: the initial state assignment, t(0), does not correspond to the state variable x

In [5]:
@system(x' = -x, x(0) ∈ Interval(-1.0, 1.0))

InitialValueProblem{LinearContinuousSystem{Int64,IdentityMultiple{Int64}},Interval{Float64,IntervalArithmetic.Interval{Float64}}}(LinearContinuousSystem{Int64,IdentityMultiple{Int64}}([-1]), Interval{Float64,IntervalArithmetic.Interval{Float64}}([-1, 1]))

In [65]:
# continuous ivp in floating-point
s = IVP(LinearContinuousSystem(I(-1.0, 1)), Interval(-1, 1))
@test @system(x' = -1.0x, x(0) ∈ Interval(-1, 1)) == s

# similar for integers
s = IVP(LinearDiscreteSystem(I(-1, 1)), [1])
@test @system(x⁺ = -x, x(0) ∈ [1]) == s

# initial state assignment doesn't match state variable
@test_throws ArgumentError @system(x' = -x, t(0) ∈ Interval(-1.0, 1.0))

Test Passed
      Thrown: ArgumentError